# Packages and Helper Functions

In [ ]:
import subprocess
import sys

try:
    from drn import *
except ImportError:
    print("The 'drn' package is not installed. Installing it now...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "git+https://github.com/EricTianDong/drn.git"])
    from drn import *

print("The 'drn' package is installed and ready to use.")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

from pathlib import Path

import scipy
from scipy.stats import wilcoxon
import statsmodels.api as sm

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm.auto import trange
import torch
import torch.nn as nn

torch.set_num_threads(1)
#from sklearn import set_config
#set_config(transform_output="pandas")

import skopt
import skopt.plots
from skopt.space import Integer, Real, Categorical
from skopt.plots import plot_convergence, plot_objective, plot_evaluations

# sns.set_style("whitegrid", {'axes.grid' : False})
plt.rcParams["savefig.dpi"] = 300

In [ ]:
# Adjust if needed
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15

In [ ]:
# The notebook will save files into subdirectories of the current working directory.
# Here we make the directories if they do not already exist.
directories = ["plots", "models/reg", "models/synth", "models/real"]
for directory in directories:
    Path(directory).mkdir(parents=True, exist_ok=True)

## Hyperparameter Tuning

In [ ]:
# Objective function to minimise for DRN
def objective_drn(params, X_train, Y_train, X_val, Y_val,
                  train_dataset, val_dataset, 
                  kl_direction='forwards', criteria='CRPS', patience=30):
    num_hidden_layers, hidden_size, dropout_rate, lr, kl_alpha, mean_alpha, dv_alpha, batch_size, proportion, min_obs = params

    # Print out the current parameters line-by-line with the name beforehand
    names = "num_hidden_layers, hidden_size, dropout_rate, lr, kl_alpha, mean_alpha, dv_alpha, batch_size, proportion, min_obs".split(", ")
    for name, val in zip(names, params):
        print(f"\tTrying {name} = {val} (type {type(val)}")

    # Since those integer values are numpy.int64, and that breaks some things, manually convert to Python ints
    num_hidden_layers = int(num_hidden_layers)
    hidden_size = int(hidden_size)
    batch_size = int(batch_size)
    min_obs = int(min_obs)

    if torch.cuda.is_available():
        device = torch.device("cuda:0")
    else:
        device = torch.device("cpu")

    cutpoints = drn_cutpoints(c_0 = np.min(Y_train.detach().numpy()) * 1.05 if np.min(Y_train.detach().numpy()) < 0 else 0.0,
                              c_K = np.max(Y_train.detach().numpy()) * 1.05,
                              p = proportion,
                              y = Y_train.detach().numpy(),
                              min_obs = min_obs)
    torch.manual_seed(23)
    drn_model = DRN(num_features = X_train.shape[1], cutpoints = cutpoints, glm = glm,\
                hidden_size=hidden_size, num_hidden_layers=num_hidden_layers, dropout_rate = dropout_rate)
    
    # Train the model with the provided hyperparameters
    try:
        torch.manual_seed(23)
        train(
            model=drn_model,
            criterion=lambda pred, y: drn_loss(pred, y, kl_alpha = kl_alpha, mean_alpha = mean_alpha,
                                                     tv_alpha = 0, dv_alpha = dv_alpha, kl_direction = kl_direction),
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            batch_size=batch_size,
            epochs=2000,
            patience=patience,
            lr=lr,
            print_details=True,
            log_interval=30,
            device=device
        )
    except Exception as e:
        print(f"Training failed: {e}")
        return 1e6  # Return a large number in case of failure

    # Calculate validation loss and return it
    grid_size = 3000  # Increase this to get more accurate CRPS estimates
    grid = torch.linspace(0, np.max(Y_train.detach().numpy()) * 1.1, grid_size).unsqueeze(-1).to(device)

    drn_model.eval()
    with torch.no_grad():
        dists = drn_model.distributions(X_val)
        cdfs = dists.cdf(grid)
        grid = grid.squeeze()
        crps_score = crps(Y_val, grid, cdfs).mean().item()
        nll_score = -dists.log_prob(Y_val).mean().item()
        nll_score = (nll_score if np.exp(-nll_score) > 0 else 1e10)
    
    print(f"CRPS: {crps_score}, NLL: {nll_score}")

    return(crps_score if criteria == 'CRPS' else nll_score)

In [ ]:
# Objective function to minimize for CANN
def objective_cann(params, X_train, Y_train, X_val, Y_val, 
                   train_dataset, val_dataset, distribution,
                   patience = 50):
    num_hidden_layers, hidden_size, dropout_rate, lr, batch_size = params
  
    names = "num_hidden_layers, hidden_size, dropout_rate, lr, batch_size".split(", ")
    for name, val in zip(names, params):
        print(f"\tTrying {name} = {val} (type {type(val)}")

    num_hidden_layers = int(num_hidden_layers)
    hidden_size = int(hidden_size)
    batch_size = int(batch_size)

    if torch.cuda.is_available():
        device = torch.device("cuda:0")
    else:
        device = torch.device("cpu")
        
    torch.manual_seed(23)
    cann = CANN(glm, num_hidden_layers=num_hidden_layers, hidden_size=hidden_size, dropout_rate = dropout_rate)

    try:
        torch.manual_seed(23)
        train(
            cann,
            gaussian_deviance_loss if distribution == "gaussian" else gamma_deviance_loss,
            train_dataset,
            val_dataset,
            epochs=2000,
            lr=lr,
            patience=patience,
            batch_size=batch_size,
        )
        cann.update_dispersion(X_train, Y_train)
    except Exception as e:
        print(f"Training failed: {e}")
        return 1e6  

    grid_size = 3000 
    grid = torch.linspace(0, np.max(Y_train.detach().numpy()) * 1.1, grid_size).unsqueeze(-1).to(device)

    cann.eval()
    with torch.no_grad():
        dists = cann.distributions(X_val)
        cdfs = dists.cdf(grid)
        grid = grid.squeeze()
        crps_score = crps(Y_val, grid, cdfs).mean().item()

    return crps_score

In [ ]:
# Objective function to minimise for MDN
def objective_mdn(params, X_train, Y_train, X_val, Y_val,
                   train_dataset, val_dataset, distribution,
                   patience = 50):
    num_hidden_layers, hidden_size, dropout_rate, lr, num_components, batch_size = params
  
    names = "num_hidden_layers, hidden_size, dropout_rate, lr, num_components, batch_size".split(", ")
    for name, val in zip(names, params):
        print(f"\tTrying {name} = {val} (type {type(val)}")

    num_hidden_layers = int(num_hidden_layers)
    hidden_size = int(hidden_size)
    batch_size = int(batch_size)
    num_components = int(num_components)

    if torch.cuda.is_available():
        device = torch.device("cuda:0")
    else:
        device = torch.device("cpu")

    torch.manual_seed(23)
    mdn = MDN(X_train.shape[1], num_components=num_components, hidden_size=hidden_size,
                   num_hidden_layers=num_hidden_layers, dropout_rate = dropout_rate,\
                distribution= distribution)
        
    try:
        torch.manual_seed(23)
        train(
                mdn,
                gaussian_mdn_loss if distribution == "gaussian" else gamma_mdn_loss,
                train_dataset,
                val_dataset,
                lr=lr,
                batch_size=batch_size,
                epochs=2000,
                patience=patience,
                device = device
        )
        mdn.eval()
        
    except Exception as e:
        print(e)
        print(f"Training failed: {e}")
        return 1e6  

    grid_size = 3000  
    grid = torch.linspace(0, np.max(Y_train.detach().numpy()) * 1.1, grid_size).unsqueeze(-1).to(device)

    mdn.eval()
    with torch.no_grad():
        dists = mdn.distributions(X_val)
        cdfs = dists.cdf(grid)
        grid = grid.squeeze()
        crps_score = crps(Y_val, grid, cdfs).mean().item()

    return crps_score

In [ ]:
# Objective function to minimise for DDR
def objective_ddr(params, X_train, Y_train, X_val, Y_val,
                   train_dataset, val_dataset,
                   patience = 30):
    num_hidden_layers, hidden_size, dropout_rate, lr, proportion, batch_size = params
  
    names = "num_hidden_layers, hidden_size, dropout_rate, lr, proportion, batch_size".split(", ")
    for name, val in zip(names, params):
        print(f"\tTrying {name} = {val} (type {type(val)}")

    num_hidden_layers = int(num_hidden_layers)
    hidden_size = int(hidden_size)
    batch_size = int(batch_size)

    if torch.cuda.is_available():
        device = torch.device("cuda:0")
    else:
        device = torch.device("cpu")
        
    cutpoints_DDR = ddr_cutpoints(c_0 = np.min(Y_train.detach().numpy()) * 1.05 if np.min(Y_train.detach().numpy()) < 0 else 0.0,
                              c_K = np.max(Y_train.detach().numpy()) * 1.05, 
                              y = Y_train.detach().numpy(),
                              p = proportion)

    torch.manual_seed(23)
    ddr_model = DDR(X_train.shape[1], cutpoints_DDR , num_hidden_layers=num_hidden_layers,\
               hidden_size=hidden_size, dropout_rate = dropout_rate)

    try:
        torch.manual_seed(23)
        train(
            ddr_model,
            ddr_loss,
            train_dataset,
            val_dataset,
            epochs=2000,
            lr=lr,
            patience=patience,
            batch_size=batch_size,
        )
    except Exception as e:
        print(f"Training failed: {e}")
        return 1e6  

    grid_size = 3000  
    grid = torch.linspace(0, np.max(Y_train.detach().numpy()) * 1.1, grid_size).unsqueeze(-1).to(device)

    ddr_model.eval()
    with torch.no_grad():
        dists = ddr_model.distributions(X_val)
        cdfs = dists.cdf(grid)
        grid = grid.squeeze()
        crps_score = crps(Y_val, grid, cdfs).mean().item()

    return crps_score

## Quantile Residuals and Calibration

In [ ]:
def quantile_residuals(y, F_, interval):
    if y < interval[0]:
        return 0
    if y > interval[len(interval) - 1]:
        return 1
    for i in range(len(interval) - 1):
        if y > interval[i] and y <= interval[i + 1]:
            idx_low = i
            idx_up = i + 1
            return 0.5 * (F_[idx_low] + F_[idx_up])

def quantile_points(cdfs, response, grid):
    response = np.array(response)
    GLM_points = [[0]] * len(response)
    CANN_points = [[0]] * len(response)
    MDN_points = [[0]] * len(response)
    DDR_points = [[0]] * len(response)
    DRN_points = [[0]] * len(response)

    for k in trange(len(response)):
        # GLM
        GLM_points[k] = quantile_residuals(response[k], cdfs["GLM"][:, k].detach().numpy(), grid.detach().numpy())

        # CANN
        CANN_points[k] = quantile_residuals(response[k], cdfs["CANN"][:, k].detach().numpy(), grid.detach().numpy())

        # MDN
        MDN_points[k] = quantile_residuals(response[k], cdfs["MDN"][:, k].detach().numpy(), grid.detach().numpy())

        # DDR
        DDR_points[k] = quantile_residuals(response[k], cdfs["DDR"][:, k].detach().numpy(), grid.detach().numpy())

        # DRN
        DRN_points[k] = quantile_residuals(response[k], cdfs["DRN"][:, k].detach().numpy(), grid.detach().numpy())

    return(GLM_points, MDN_points, DDR_points, DRN_points, CANN_points)

def quantile_residuals_plots(model_points):
    quantiles = [0] * len(model_points)
    for i in range(len(model_points)):
        quantiles[i] = np.array(scipy.stats.norm.ppf(model_points[i]))

    figure, axes = plt.subplots(2, 2, figsize=(26, 26))
    sm.qqplot(quantiles[0], line="45", ax=axes[0, 0])
    sm.qqplot(quantiles[1], line="45", ax=axes[0, 1])
    sm.qqplot(quantiles[2], line="45", ax=axes[1, 0])
    sm.qqplot(quantiles[3], line="45", ax=axes[1, 1])
    # sm.qqplot(quantiles[4], line="45", ax=axes[2, 0])
    axes[0, 0].set_title("GLM", fontsize=45, color="black")
    axes[0, 0].set_ylim(-4, 4)
    axes[0, 0].set_xlim(-3.2, 3.2)
    axes[0, 1].set_title("MDN", fontsize=45, color="black")
    axes[0, 1].set_ylim(-4, 4)
    axes[0, 1].set_xlim(-3.2, 3.2)
    axes[1, 0].set_title("DDR", fontsize=45, color="black")
    axes[1, 0].set_ylim(-4, 4)
    axes[1, 0].set_xlim(-3.2, 3.2)
    axes[1, 1].set_title("DRN", fontsize=45, color="black")
    axes[1, 1].set_ylim(-4, 4)
    axes[1, 1].set_xlim(-3.2, 3.2)
    
    # Set font size for all axes labels and tick labels
    for ax in axes.flat:
        # Set the font size of axis labels
        ax.set_xlabel('Theoretical Quantiles', fontsize=45)  # Adjust fontsize as needed
        ax.set_ylabel('Sample Quantiles', fontsize=45)  # Adjust fontsize as needed
        
        # Set the font size of tick labels
        ax.tick_params(axis='both', which='major', labelsize=40)  # Adjust labelsize as needed

    figure.suptitle("Quantile Residuals", fontsize=60, y=0.99) #fontweight="bold"
    plt.tight_layout(pad=2)

In [ ]:
# Find the index (from the list lst_new) that gives the closest value to the given scalar y
def closest_index(y, lst_new):
    low, high = 0, len(lst_new) - 1
    while low < high - 1:
        mid = (low + high) // 2
        if lst_new[mid] == y:
            return mid
        elif lst_new[mid] < y:
            low = mid
        else:
            high = mid

    if abs(lst_new[low] - y) <= abs(lst_new[high] - y):
        return low
    else:
        return high

def calibration_plot_stats(cdfs_, grid, responses):
    Q_predicted = [[0]] * len(responses)
    Q_empirical = [[0]] * len(responses)

    cdfs_ = cdfs_.T

    for k in trange(len(responses)):
        y = responses[k]
        Q_predicted[k] = np.array(cdfs_[k])[closest_index(y, grid)]
        all_quantiles = [
            np.array(cdfs_[j])[closest_index(responses[j], grid)] <= Q_predicted[k]
            for j in range(len(responses))
        ]
        Q_empirical[k] = np.sum(all_quantiles) / len(responses)

    return (Q_predicted, Q_empirical)

def calibration_plot(cdfs_, y, grid):
    responses = np.array(y)

    Q_predicted_GLM, Q_empirical_GLM = calibration_plot_stats(
        cdfs_["GLM"].detach().numpy(), grid.detach().numpy(), responses
    )
    Q_predicted_CANN, Q_empirical_CANN = calibration_plot_stats(
        cdfs_["CANN"].detach().numpy(), grid.detach().numpy(), responses
    )
    Q_predicted_MDN, Q_empirical_MDN = calibration_plot_stats(
        cdfs_["MDN"].detach().numpy(), grid.detach().numpy(), responses
    )
    Q_predicted_DDR, Q_empirical_DDR = calibration_plot_stats(
        cdfs_["DDR"].detach().numpy(), grid.detach().numpy(), responses
    )
    Q_predicted_DRN, Q_empirical_DRN = calibration_plot_stats(
        cdfs_["DRN"].detach().numpy(), grid.detach().numpy(), responses
    )

    GLM_STATS = np.sum((np.array(Q_predicted_GLM) - np.array(Q_empirical_GLM)) ** 2) / len(
        responses
    )
    CANN_STATS = np.sum(
        (np.array(Q_predicted_CANN) - np.array(Q_empirical_CANN)) ** 2
    ) / len(responses)
    MDN_STATS = np.sum((np.array(Q_predicted_MDN) - np.array(Q_empirical_MDN)) ** 2) / len(
        responses
    )
    DDR_STATS = np.sum((np.array(Q_predicted_DDR) - np.array(Q_empirical_DDR)) ** 2) / len(
        responses
    )
    DRN_STATS = np.sum(
        (np.array(Q_predicted_DRN) - np.array(Q_empirical_DRN)) ** 2
    ) / len(responses)

    figure, axes = plt.subplots(1, 1, figsize=(10, 10))

    plt.scatter(
        Q_predicted_GLM,
        Q_empirical_GLM,
        s=6,
        color="gray",
        label=f"GLM \n $\sum_j (p_j-\hat p_j)^2= {round(GLM_STATS*len(responses), 4)}$",
    )
    plt.scatter(
        Q_predicted_CANN,
        Q_empirical_CANN,
        s=6,
        color="green",
        label=f"CANN \n $\sum_j (p_j-\hat p_j)^2= {round(CANN_STATS*len(responses), 4)}$",
    )
    plt.scatter(
        Q_predicted_MDN,
        Q_empirical_MDN,
        s=6,
        color="black",
        label=f"MDN \n $\sum_j (p_j-\hat p_j)^2= {round(MDN_STATS*len(responses), 4)}$",
    )
    plt.scatter(
        Q_predicted_DDR,
        Q_empirical_DDR,
        s=6,
        label=f"DDR \n $\sum_j (p_j-\hat p_j)^2= {round(DDR_STATS*len(responses), 4)}$",
    )
    plt.scatter(
        Q_predicted_DRN,
        Q_empirical_DRN,
        s=6,
        color="red",
        label=f"DRN  \n $\sum_j (p_j- p_j)^2={round(DRN_STATS*len(responses), 4)}$",
    )
    plt.plot([0, 1], [0, 1], ls="--", color="red")
    plt.xlabel("Predicted: $\hat{p}$", fontsize=30)
    plt.ylabel("Empirical: $p$", fontsize=30)
    plt.title("Calibration Plot", fontsize=30)
    legend = plt.legend(prop={"size": 15}, scatterpoints=1)  # Increase scatterpoints for larger marker
    for handle in legend.legend_handles:
        handle.set_sizes([40]) 

## Wilcoxon Test

In [ ]:
def print_wilcoxon_test(glm_metrics, cann_metrics, mdn_metrics, ddr_metrics, drn_metrics):
    # Perform the Wilcoxon Signed-Rank Test
    stat, p_value = wilcoxon(drn_metrics, glm_metrics, alternative='less')
    print("DRN < GLM")
    print("Wilcoxon Signed-Rank Test statistic:", stat)
    print("P-value:", p_value)

    stat, p_value = wilcoxon(drn_metrics, cann_metrics, alternative='less')
    print("DRN < CANN")
    print("Wilcoxon Signed-Rank Test statistic:", stat)
    print("P-value:", p_value)

    stat, p_value = wilcoxon(drn_metrics, mdn_metrics, alternative='less')
    print("DRN < MDN")
    print("Wilcoxon Signed-Rank Test statistic:", stat)
    print("P-value:", p_value)

    stat, p_value = wilcoxon(drn_metrics, ddr_metrics, alternative='less')
    print("DRN < DDR")
    print("Wilcoxon Signed-Rank Test statistic:", stat)
    print("P-value:", p_value)

In [ ]:
def nll_wilcoxon_test(dists, Y_target, dataset = 'Test'):
    # NLL data 
    nll_model_glm = -dists['GLM'].log_prob(Y_target).squeeze().detach().numpy() 
    nll_model_cann = -dists['CANN'].log_prob(Y_target).squeeze().detach().numpy() 
    nll_model_mdn = -dists['MDN'].log_prob(Y_target).squeeze().detach().numpy() 
    nll_model_ddr = -dists['DDR'].log_prob(Y_target).squeeze().detach().numpy() 
    nll_model_drn = -dists['DRN'].log_prob(Y_target).squeeze().detach().numpy() 

    print("--------------------------------------------")
    print(f"{dataset} Data")
    print("--------------------------------------------")

    print_wilcoxon_test(nll_model_glm, nll_model_cann, nll_model_mdn, nll_model_ddr, nll_model_drn)

In [ ]:
def crps_wilcoxon_test(cdfs_, Y_target, grid, dataset = 'Test'):
    # CRPS data 
    crps_model_drn =  crps(Y_target, grid, cdfs_['DRN']).squeeze().detach().numpy() 
    crps_model_glm =  crps(Y_target, grid, cdfs_['GLM']).squeeze().detach().numpy() 
    crps_model_cann =  crps(Y_target, grid, cdfs_['CANN']).squeeze().detach().numpy() 
    crps_model_mdn =  crps(Y_target, grid, cdfs_['MDN']).squeeze().detach().numpy() 
    crps_model_ddr =  crps(Y_target, grid, cdfs_['DDR']).squeeze().detach().numpy() 

    print("--------------------------------------------")
    print(f"{dataset} Data")
    print("--------------------------------------------")

    print_wilcoxon_test(crps_model_glm, crps_model_cann, crps_model_mdn, crps_model_ddr, crps_model_drn)
    

In [ ]:
def rmse_wilcoxon_test(dists_, Y_target, dataset = 'Test'):
    # MSE data 
    se_drn =  (dists_['DRN'].mean.squeeze().detach().numpy() - Y_target.squeeze().detach().numpy())**2
    se_glm =  (dists_['GLM'].mean.squeeze().detach().numpy() - Y_target.squeeze().detach().numpy())**2
    se_cann =  (dists_['CANN'].mean.squeeze().detach().numpy() - Y_target.squeeze().detach().numpy())**2
    se_mdn =  (dists_['MDN'].mean.squeeze().detach().numpy() - Y_target.squeeze().detach().numpy())**2
    se_ddr =  (dists_['DDR'].mean.squeeze().detach().numpy() - Y_target.squeeze().detach().numpy())**2

    print("--------------------------------------------")
    print(f"{dataset} Data")
    print("--------------------------------------------")

    print_wilcoxon_test(se_glm, se_cann, se_mdn, se_ddr, se_drn)

In [ ]:

def quantile_score_raw(y_true, y_pred, p):
    """
    Compute the quantile score for predictions at a specific quantile.

    :param y_true: Actual target values as a Pandas Series or PyTorch tensor.
    :param y_pred: Predicted target values as a numpy array or PyTorch tensor.
    :param p: The cumulative probability as a float
    :return: The quantile score as a PyTorch tensor.
    """
    # Ensure that y_true and y_pred are PyTorch tensors
    y_true = (
        torch.Tensor(y_true.values) if not isinstance(y_true, torch.Tensor) else y_true
    )
    y_pred = torch.Tensor(y_pred) if not isinstance(y_pred, torch.Tensor) else y_pred
    # Reshape y_pred to match y_true if necessary and compute the error
    e = y_true - y_pred.reshape(y_true.shape)
    # Compute the quantile score
    return torch.where(y_true >= y_pred, p * e, (1 - p) * -e)


def quantile_losses_raw(
    p,
    model,
    model_name,
    X,
    y,
    max_iter=1000,
    tolerance=5e-5,
    l=None,
    u=None,
    print_score=True,
):
    """
    Calculate and optionally print the quantile loss for the given data and model.

    :param p: The cumulative probability ntile as a float
    :param model: The trained model.
    :param model_name: The name of the trained model.
    :param X: Input features as a Pandas DataFrame or numpy array.
    :param y: True target values as a Pandas Series or numpy array.
    :param max_iter: The maximum number of iterations for the quantile search algorithm.
    :param tolerance: The tolerance for convergence of the the quantile search algorithm.
    :param l: The lower bound for the quantile search
    :param u: The upper bound for the quantile search
    :param print_score: A boolean indicating whether to print the score.
    :return: The quantile loss as a PyTorch tensor.
    """
    # Predict quantiles based on the model name
    if model_name in ["GLM", "MDN", "CANN"]:
        predicted_quantiles = model.quantiles(
            X, [p * 100], max_iter=max_iter, tolerance=tolerance, l=l, u=u
        )
    elif model_name in ["DDR", "DRN"]:
        predicted_quantiles = model.distributions(X).quantiles(
            [p * 100], max_iter=max_iter, tolerance=tolerance, l=l, u=u
        )

    # Compute the quantile score
    score = quantile_score_raw(y, predicted_quantiles, p)

    return score

In [ ]:
def ql90_wilcoxon_test(X_features, Y_target, dataset = 'Test'):
    # 90% QL data 
    ql_glm =  quantile_losses_raw(0.9, glm, 'GLM', X_features, Y_target,
                                        max_iter = 1000, tolerance = 1e-4,\
                                        l = torch.Tensor([0]),\
                                        u = torch.Tensor([np.max(y_train)+3*(np.max(y_train)-np.min(y_train))])).squeeze().detach().numpy()
    ql_glm =  quantile_losses_raw(0.9, glm, 'CANN', X_features, Y_target,
                                        max_iter = 1000, tolerance = 1e-4,\
                                        l = torch.Tensor([0]),\
                                        u = torch.Tensor([np.max(y_train)+3*(np.max(y_train)-np.min(y_train))])).squeeze().detach().numpy()
    ql_mdn =  quantile_losses_raw(0.9, mdn, 'MDN', X_features, Y_target,
                                        max_iter = 1000, tolerance = 1e-4,\
                                        l = torch.Tensor([0]),\
                                        u = torch.Tensor([np.max(y_train)+3*(np.max(y_train)-np.min(y_train))])).squeeze().detach().numpy()
    ql_ddr =  quantile_losses_raw(0.9, ddr, 'DDR', X_features, Y_target,
                                        max_iter = 1000, tolerance = 1e-4,\
                                        l = torch.Tensor([0]),\
                                        u = torch.Tensor([np.max(y_train)+3*(np.max(y_train)-np.min(y_train))])).squeeze().detach().numpy()
    ql_drn =  quantile_losses_raw(0.9, drn, 'DRN', X_features, Y_target,
                                        max_iter = 1000, tolerance = 1e-4,\
                                        l = torch.Tensor([0]),\
                                        u = torch.Tensor([np.max(y_train)+3*(np.max(y_train)-np.min(y_train))])).squeeze().detach().numpy()

    print("--------------------------------------------")
    print(f"{dataset} Data")
    print("--------------------------------------------")

    print_wilcoxon_test(ql_glm, ql_glm, ql_mdn, ql_ddr, ql_drn)

# Section 4: Regularisation

In [ ]:
def generate_synthetic_gaussian(n=1000, seed=1, specific_instance = None):
    rng = np.random.default_rng(seed)
    # Parameters
    mu = [0, 0]  # means
    sigma = [0.5, 0.5]  # standard deviations
    rho = 0.0  # correlation coefficient

    # Covariance matrix
    covariance = [
        [sigma[0] ** 2, rho * sigma[0] * sigma[1]],
        [rho * sigma[0] * sigma[1], sigma[1] ** 2],
    ]

    # Generate bivariate normal distribution
    x = rng.multivariate_normal(mu, covariance, n)

    # Create a non-linear and non-stationary relationship between X_1, X_2 and Y
    means = (- x[:, 0] +  x[:, 1]) #+ 0.2 * x[:, 1]**2
    dispersion = 0.5 * (x[:, 0] ** 2 + x[:, 1] ** 2) 

    if specific_instance is not None:
        x_1 = specific_instance[0]
        x_2 = specific_instance[1]
        means = (- x_1 + x_2).repeat(n)
        dispersion = (0.5 * (x_1**2 + x_2**2)).repeat(n)

    y_normal = rng.normal(means, dispersion)

    # Combine the components    
    y = (y_normal) 

    return pd.DataFrame(x, columns=["X_1", "X_2"]), pd.Series(y, name="Y"), means, dispersion

In [ ]:
features, target, means, dispersion = generate_synthetic_gaussian(40000)
x_train, x_val, x_test, y_train, y_val, y_test,\
      x_train_raw, x_val_raw, x_test_raw,\
          num_features, cat_features,\
             all_categories, ct =\
                split_and_preprocess(features, target, ['X_1', 'X_2'], [], seed = 0, num_standard = False)

In [ ]:
# We'll use lowercase "x_train" "y_train" for pandas dataframes
# and uppercase "X_train" "Y_train" for tensors.
X_train = torch.Tensor(x_train.values)
Y_train = torch.Tensor(y_train.values)
X_val = torch.Tensor(x_val.values)
Y_val = torch.Tensor(y_val.values)
X_test = torch.Tensor(x_test.values)
Y_test = torch.Tensor(y_test.values)

train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, Y_val)

In [ ]:
training = False 
distribution = "gaussian" # distributional assumption for the GLM, CANN, MDN

## Baseline

In [ ]:
print("\n\nTraining GLM\n")
glm = GLM.from_statsmodels(X_train, Y_train, distribution=distribution)
torch.save(glm.state_dict(), "models/reg/glm.pt")

In [ ]:
cutpoints_DRN = drn_cutpoints(c_0 = np.min(y_train) * 1.05 if np.min(y_train) < 0 else 0.0,
                              c_K = np.max(y_train) * 1.05,
                              p = 0.01,
                              y = y_train,
                              min_obs = 1)

## (a) No Regularisation

In [ ]:
print("\n\nTraining DRN\n")
torch.manual_seed(23)
drn_no_penalty = DRN(num_features = x_train.shape[1], cutpoints = cutpoints_DRN, glm = glm,
     hidden_size=100, num_hidden_layers=2, baseline_start = False, dropout_rate = 0.2)
if training:
    train(
        drn_no_penalty,
        lambda pred, y : drn_loss(pred, y, kl_alpha = 0,
                                                mean_alpha = 0,
                                                dv_alpha = 0,
                                                tv_alpha = 0),   
        train_dataset,
        val_dataset, 
        lr=0.001, #lr = 0.0002
        batch_size= 200, #batch_size = 50
        log_interval=1,
        patience=30,
        epochs=1000,
    )
    drn_no_penalty.eval()
    torch.save(drn_no_penalty.state_dict(), "models/reg/drn_no_penalty.pt")
else:
    drn_no_penalty.load_state_dict(torch.load("models/reg/drn_no_penalty.pt"))
    drn_no_penalty.eval()

In [ ]:
x_1 = 0.5
x_2 = 0.5
instance = pd.DataFrame(np.array([x_1, x_2]).reshape(1,2), columns = ['X_1', 'X_2'])

true_mean = -x_1 + x_2
true_scale = 0.5 * (x_1**2 + x_2**2)

## (b) Small KL

In [ ]:
print("\n\nTraining DRN\n")
torch.manual_seed(23)
drn_kl_penalty = DRN(num_features = x_train.shape[1], cutpoints = cutpoints_DRN, glm = glm,\
     hidden_size=128, num_hidden_layers=2, baseline_start = False,  dropout_rate = 0.2)
if training:
    train(
        drn_kl_penalty,
        lambda pred, y : drn_loss(pred, y, kl_alpha = 0.001,
                                                mean_alpha = 0,  #5e-2
                                                dv_alpha = 0, #5e-4
                                                tv_alpha = 0),   
        train_dataset,
        val_dataset, 
        lr=0.001, #lr = 0.0002
        batch_size= 200, #batch_size = 50
        log_interval=1,
        patience=10,
        epochs=1000,
    )
    drn_kl_penalty.eval()
    torch.save(drn_kl_penalty.state_dict(), "models/reg/drn_kl_penalty.pt")
else:
    drn_kl_penalty.load_state_dict(torch.load("models/reg/drn_kl_penalty.pt"))
    drn_kl_penalty.eval()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3.5))

# Calculate the PDF with 'true_mean' and 'true_scale' parameters
true_y_grid = np.linspace(-1.25, 1.25, 1000)
true_densities = scipy.stats.norm.pdf(true_y_grid, loc=true_mean, scale=true_scale)

# Create the Explainer class to generate plots
drn_no_penalty_exp = DRNExplainer(drn_no_penalty, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, ct)  
drn_no_penalty_exp.plot_adjustment_factors(instance = instance,\
                                           num_interpolations=1_000, 
                                           plot_adjustments_labels = False,\
                                            axes = ax1,
                                            x_range = (-1.25, 1.25),\
                                            y_range = (0, 2),\
                                            plot_title = "",
                                            plot_y_label = "$f(y|\\boldsymbol{X}=(0.5, 0.5)^{\\top})$", 
                                            )

ax1.plot(true_y_grid, true_densities, color='red', label='True', lw=2, zorder=-1)
plt.legend()
sns.despine()

drn_kl_penalty_exp = DRNExplainer(drn_kl_penalty, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, ct)  
drn_kl_penalty_exp.plot_adjustment_factors(instance = instance,\
                                          num_interpolations=1_000, 
                                          plot_adjustments_labels = False,\
                                          axes = ax2,
                                          x_range = (-1.25, 1.25),\
                                          y_range = (0, 2),\
                                            plot_title = "",
                                           plot_y_label = "$f(y|\\boldsymbol{X}=(0.5, 0.5)^{\\top})$", 
                                            )

ax2.plot(true_y_grid, true_densities, color='red', label='True', lw=2, zorder=-1)
plt.legend()
sns.despine()

plt.tight_layout()
plt.savefig("plots/DRN KL Penalty.png");

## (c) Excessive Smoothing

In [ ]:
print("\n\nTraining DRN\n")
torch.manual_seed(23)
drn_dv_large_penalty = DRN(num_features = x_train.shape[1], cutpoints = cutpoints_DRN, glm = glm,\
     hidden_size=128, num_hidden_layers=2, baseline_start = True,  dropout_rate = 0.2)
if training:
    train(
        drn_dv_large_penalty,
        lambda pred, y : drn_loss(pred, y, kl_alpha = 0, 
                                            mean_alpha = 0,  
                                            dv_alpha = 10, 
                                            tv_alpha = 0),   
        train_dataset,
        val_dataset, 
        lr=0.01, 
        batch_size= 300,
        log_interval=1,
        patience=10,
        epochs=1000,
    )
    drn_dv_large_penalty.eval()
    torch.save(drn_dv_large_penalty.state_dict(), "models/reg/drn_dv_large_penalty.pt")
else:
    drn_dv_large_penalty.load_state_dict(torch.load("models/reg/drn_dv_large_penalty.pt"))
    drn_dv_large_penalty.eval()

## (d) Perfect Smoothing

In [ ]:
print("\n\nTraining DRN\n")
torch.manual_seed(23)
drn_everything = DRN(num_features = x_train.shape[1], cutpoints = cutpoints_DRN, glm = glm,\
     hidden_size=128, num_hidden_layers=2, baseline_start = False,  dropout_rate = 0.2)
if training:
    train(
        drn_everything,
        lambda pred, y : drn_loss(pred, y, kl_alpha = 1e-3, 
                                                mean_alpha = 0,  
                                                dv_alpha = 5e-4,
                                                tv_alpha = 0),   
        train_dataset,
        val_dataset, 
        lr=0.001, 
        batch_size= 100, 
        log_interval=1,
        patience=10,
        epochs=1000,
    )
    drn_everything.eval()
    torch.save(drn_everything.state_dict(), "models/reg/drn_everything.pt")
else:
    drn_everything.load_state_dict(torch.load("models/reg/drn_everything.pt"))
    drn_everything.eval()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3.5))

# Calculate the PDF with 'true_mean' and 'true_scale' parameters
true_y_grid = np.linspace(-1.25, 1.25, 1000)
true_densities = scipy.stats.norm.pdf(true_y_grid, loc=true_mean, scale=true_scale)

drn_explainer = DRNExplainer(drn_dv_large_penalty, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, ct)  
drn_explainer.plot_adjustment_factors(instance = instance,\
                                      num_interpolations=1_000, 
                                        plot_adjustments_labels = False,\
                                          axes = ax1,
                                          x_range = (-1.25, 1.25),\
                                          y_range = (0, 2),\
                                          plot_title = "",
                                           plot_y_label = "$f(y|\\boldsymbol{X}=(0.5, 0.5)^{\\top})$", 
                                            )

ax1.plot(true_y_grid, true_densities, color='red', label='True', lw=2, zorder=-1)
plt.legend()
sns.despine()

drn_explainer = DRNExplainer(drn_everything, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, ct)  
drn_explainer.plot_adjustment_factors(instance = instance,\
                                      num_interpolations=1_000, 
                                        plot_adjustments_labels = False,\
                                          axes = ax2,
                                          x_range = (-1.25, 1.25),\
                                          y_range = (0, 2),\
                                            plot_title = "",
                                           plot_y_label = "$f(y|\\boldsymbol{X}=(0.5, 0.5)^{\\top})$", 
                                            )

ax2.plot(true_y_grid, true_densities, color='red', label='True', lw=2, zorder=-1)
plt.legend()
sns.despine()

plt.tight_layout()
plt.savefig("plots/DRN DV Penalty.png");

# Section 5: Synthetic Data

## Sec 5.1: Data Generation

In [ ]:
def generate_synthetic_gamma_lognormal(n=1000, seed=1, specific_instance = None):
    rng = np.random.default_rng(seed)
    # Parameters
    mu = [0, 0]  # means
    sigma = [0.25, 0.25]  # standard deviations
    rho = 0.25  # correlation coefficient

    # Covariance matrix
    covariance = [
        [sigma[0] ** 2, rho * sigma[0] * sigma[1]],
        [rho * sigma[0] * sigma[1], sigma[1] ** 2],
    ]

    # Generate bivariate normal distribution
    x = rng.multivariate_normal(mu, covariance, n)

    # Create a non-linear and non-stationary relationship between X_1, X_2 and Y
    means = np.exp(- x[:, 0] +  x[:, 1]) 
    dispersion = np.exp(x[:, 0])  / (1 + np.exp((x[:, 0]) * (x[:, 1])))

    if specific_instance is not None:
        x_1 = specific_instance[0]
        x_2 = specific_instance[1]
        means = np.exp(- x_1 + x_2).repeat(n)
        dispersion = (np.exp(x_1) / (1 + np.exp(x_1 * x_2))).repeat(n)

    # Calculate the gamma and lognormal parts of the Y
    y_gamma = rng.gamma(1 / dispersion, scale = dispersion * means)
    y_lognormal = np.exp(rng.normal(np.log(means), scale = dispersion))
    # Combine the components
    y = (y_gamma + y_lognormal)

    return pd.DataFrame(x, columns=["X_1", "X_2"]), pd.Series(y, name="Y"), means, dispersion

In [ ]:
features, target, means, dispersion = generate_synthetic_gamma_lognormal(20000)

In [ ]:
x_train, x_val, x_test, y_train, y_val, y_test,\
      x_train_raw, x_val_raw, x_test_raw,\
          num_features, cat_features,\
             all_categories, ct =\
                split_and_preprocess(features, target, ['X_1', 'X_2'], [], seed = 42, num_standard = False)
x_train

In [ ]:
# We'll use lowercase "x_train" "y_train" for pandas dataframes
# and uppercase "X_train" "Y_train" for tensors.
X_train = torch.Tensor(x_train.values)
Y_train = torch.Tensor(y_train.values)
X_val = torch.Tensor(x_val.values)
Y_val = torch.Tensor(y_val.values)
X_test = torch.Tensor(x_test.values)
Y_test = torch.Tensor(y_test.values)

train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, Y_val)

## Sec 5.2: Training

In [ ]:
training = False 
tuning = False # change it to True if you wish to start hyperpameter tuning
shap_run = False # change it to True if you wish to generate SHAP dependence plots 
distribution = "gamma" # distributional assumption for the GLM, CANN, MDN

### GLM

In [ ]:
%%time
print("\n\nTraining GLM\n")

torch.manual_seed(23)
glm = GLM(X_train.shape[1], distribution = distribution)
if training:
    torch.manual_seed(23)
    glm = glm.from_statsmodels(X_train, Y_train, distribution=distribution)
    glm.eval()
    torch.save(glm.state_dict(), "models/synth/glm.pt")
else:
    torch.manual_seed(23)
    glm.load_state_dict(torch.load("models/synth/glm.pt"))
    glm.eval()

### CANN

In [ ]:
if tuning:
    space_cann_synth = [
        Integer(1, 4, name='num_hidden_layers'),
        Categorical([16, 32, 64, 128, 256, 512], name='hidden_size'),
        Real(0.0, 0.5, name='dropout_rate'),
        Real(0.0002, 0.01, name='lr'),
        Categorical([128, 256, 512], name='batch_size'),
    ]

    # Run Bayesian optimization
    res_cann_synth = skopt.gp_minimize(
        lambda params: objective_cann(params, X_train, Y_train, X_val, Y_val, train_dataset, val_dataset, 'gamma'),
        space_cann_synth,
        n_calls=125,
        n_random_starts=25,
        random_state=0,
        verbose=True
    )
    best_cann_paras = res_cann_synth.x
    print(best_cann_paras)

In [ ]:
%%time
print("\n\nTraining CANN\n")

torch.manual_seed(23)

# Identical hyperparamters from the paper
if not tuning:
    # Format: [hidden layers, neurons/layer,
    #          dropout rate, learning rate, batch size]
    best_cann_paras =[3, 512, 0.0998147335740975, 0.006377183351881593, 256]

cann = CANN(glm, num_hidden_layers=int(best_cann_paras[0]), hidden_size=int(best_cann_paras[1]),
         dropout_rate = best_cann_paras[2])
if training:
    torch.manual_seed(23)
    train(
        cann,
        gaussian_deviance_loss if distribution == "gaussian" else gamma_deviance_loss,
        train_dataset,
        val_dataset,
        epochs=2000,
        lr=best_cann_paras[-2],
        patience=50,
        batch_size=int(best_cann_paras[-1]),
    )
    cann.update_dispersion(X_train, Y_train)
    cann.eval()
    torch.save(cann.state_dict(), "models/synth/cann.pt")
else:
    cann.load_state_dict(torch.load("models/synth/cann.pt"))
    cann.eval()

### MDN

In [ ]:
if tuning:
    space_mdn_synth = [
        Integer(1, 4, name='num_hidden_layers'),
        Categorical([16, 32, 64, 128, 256, 512], name='hidden_size'),
        Real(0.0, 0.5, name='dropout_rate'),
        Real(0.0002, 0.01, name='lr'),
        Integer(2, 10, name='num_components'),
        Categorical([128, 256, 512], name='batch_size'),
    ]

    # Run Bayesian optimization
    res_mdn_synth = skopt.gp_minimize(
        lambda params: objective_mdn(params,
                                    X_train,
                                    Y_train,
                                    X_val, 
                                    Y_val, 
                                    train_dataset, 
                                    val_dataset, 
                                    'gamma'),
        space_mdn_synth,
        n_calls=125,
        n_random_starts=25,
        random_state=0,
        verbose=True
    )
    best_mdn_paras = res_mdn_synth.x
    print(best_mdn_paras)

In [ ]:
%%time
print("\n\nTraining MDN\n")

# Identical hyperparameters from the paper
if not tuning:
    # Format: [hidden layers, neurons/layer,
    #          dropout rate, learning rate,
    #          components, batch size]
    best_mdn_paras = [1, 256, 0.5, 0.004506668905856003, 10, 128]

torch.manual_seed(23)
mdn = MDN(
        X_train.shape[1],
        num_hidden_layers=int(best_mdn_paras[0]),
        hidden_size=int(best_mdn_paras[1]),
        dropout_rate = best_mdn_paras[2],\
        num_components=int(best_mdn_paras[-2]),
        distribution= distribution)

if training:
    torch.manual_seed(23)
    train(
        mdn,
        gaussian_mdn_loss if distribution == "gaussian" else gamma_mdn_loss,
        train_dataset,
        val_dataset,
        lr=best_mdn_paras[3],
        batch_size=int(best_mdn_paras[-1]),
        epochs=2000,
        patience=50,
    )
    mdn.eval()
    torch.save(mdn.state_dict(), "models/synth/mdn.pt")
else:
    mdn.load_state_dict(torch.load("models/synth/mdn.pt"))
    mdn.eval()

### DDR

In [ ]:
if tuning:
    space_ddr_synth = [
        Integer(1, 4, name='num_hidden_layers'),
        Categorical([16, 32, 64, 128, 256, 512], name='hidden_size'),
        Real(0.0, 0.5, name='dropout_rate'),
        Real(0.0002, 0.01, name='lr'),
        Categorical(np.linspace(0.01, 0.03, 9), name='proportion'),
        Categorical([128, 256, 512], name='batch_size'),
    ]

    # Run Bayesian optimization
    res_ddr_synth = skopt.gp_minimize(
        lambda params: objective_ddr(params, X_train, Y_train, X_val, Y_val, train_dataset, val_dataset),
        space_ddr_synth,
        n_calls=125,
        n_random_starts=25,
        random_state=0,
        verbose=True
    )
    best_ddr_paras = res_ddr_synth.x
    print(best_ddr_paras)

In [ ]:
%%time
print("\n\nTraining DDR\n")

# Identical hyperparamters from the paper
if not tuning:
    # Format: [hidden layers, neurons/layer,
    #          dropout rate, learning rate,
    #          cutpoints proportion, batch size]
    best_ddr_paras = [3, 32, 0.019212713236367366, 0.006415885767981884, 0.03, 256]

cutpoints_DDR = ddr_cutpoints(c_0 = np.min(y_train) * 1.05 if np.min(y_train) < 0 else 0.0,
                              c_K = np.max(y_train) * 1.05, 
                              y = y_train,
                              p = best_ddr_paras[-2])

torch.manual_seed(23)
ddr = DDR(x_train.shape[1], cutpoints_DDR , num_hidden_layers=int(best_ddr_paras[0]),
                     hidden_size=int(best_ddr_paras[1]), dropout_rate = best_ddr_paras[2])
if training:
    torch.manual_seed(23)
    train(
        ddr,
        ddr_loss,
        train_dataset,
        val_dataset, 
        lr=best_ddr_paras[3],
        batch_size=int(best_ddr_paras[-1]),
        log_interval=1,
        patience=30,
        epochs=2000,
    )
    ddr.eval()
    torch.save(ddr.state_dict(), "models/synth/ddr.pt")
else:
    ddr.load_state_dict(torch.load("models/synth/ddr.pt"))
    ddr.eval()

### DRN

In [ ]:
if tuning:
    space_drn_synth = [
        Integer(1, 4, name='num_hidden_layers'),
        Categorical([16, 32, 64, 128, 256, 512], name='hidden_size'),
        Real(0.0, 0.5, name='dropout_rate'),
        Real(0.0002, 0.01, name='lr', prior = 'log-uniform'),
        Real(1e-5, 1e-1, name='kl_alpha', prior = 'log-uniform'),
        Categorical([1e-5, 1e-4, 1e-3, 1e-2, 1e-1], name='mean_alpha'),
        Categorical([1e-3, 1e-2, 1e-1], name='dv_alpha'),
        Categorical([128, 256, 512], name='batch_size'),
        Categorical(np.linspace(0.02, 0.03, 5), name='proportion'),
        Categorical([1, 3, 5], name='min_obs'),
    ]

    # Run Bayesian optimization
    res_drn_synth = skopt.gp_minimize(
        lambda params: objective_drn(params, criteria = 'CRPS', 
                                    X_train=X_train,
                                    Y_train=Y_train,
                                    X_val=X_val,
                                    Y_val=Y_val,
                                    train_dataset=train_dataset,
                                    val_dataset=val_dataset,
                                    kl_direction = 'forwards'),
        space_drn_synth,
        n_calls=125,
        n_random_starts=25,
        random_state=0,
        verbose=True
    )
    best_drn_paras = res_drn_synth.x
    print(best_drn_paras)

    # If we wish,we can generate the following 'correlation' plot,
    # loss function values vs. hyperparameters 
    with plt.rc_context(
                    {'xtick.labelsize': 'x-small', 
                     'ytick.labelsize': 'x-small',
                     'axes.labelsize': 'x-small',
                     'axes.titlesize': 'x-small'}):
        plot_objective(res_drn_synth)
        plt.show()

In [ ]:
%%time
print("\n\nTraining DRN\n")

# Identical hyperparamters from the paper
if not tuning:
  # Format: [hidden layers, neurons/layer,
  #          dropout rate, learning rate,
  #          kl penalty, mean penalty, roughnesss penalty,
  #           batch size, proportion, minimum number of observations]
  best_drn_paras = [3, 128,
                   0.13984058206702007, 0.0008092281223896455,
                   0.0004677145949629566, 0.01, 0.1,
                   256, 0.025, 5]

cutpoints_DRN = drn_cutpoints(c_0 = np.min(Y_train.detach().numpy()) * 1.05 if np.min(Y_train.detach().numpy()) < 0 else 0.0,
                              c_K = np.max(Y_train.detach().numpy()) * 1.05,
                              p = best_drn_paras[-2],
                              y = Y_train.detach().numpy(),
                              min_obs = int(best_drn_paras[-1]))
    

torch.manual_seed(23)
drn = DRN(num_features = X_train.shape[1], cutpoints = cutpoints_DRN, glm = glm,\
                    hidden_size=int(best_drn_paras[1]), num_hidden_layers=int(best_drn_paras[0]),
                      baseline_start = False,  dropout_rate = best_drn_paras[2])
if training:  
    torch.manual_seed(23)
    train(
                model=drn,
                criterion=lambda pred, y: drn_loss(pred, y, kl_alpha = best_drn_paras[4], #2e-4
                                                    mean_alpha = best_drn_paras[5],
                                                    dv_alpha = best_drn_paras[6],
                                                     kl_direction = 'forwards'),   
                train_dataset=train_dataset,
                val_dataset=val_dataset,
                batch_size=int(best_drn_paras[7]),
                epochs=2000,
                patience=30,
                lr=best_drn_paras[3],
                print_details=True,
                log_interval=1,
    )
    drn.eval()
    torch.save(drn.state_dict(), "models/synth/drn_tuned.pt")
else:
    drn.load_state_dict(torch.load("models/synth/drn_tuned.pt"))
    drn.eval()

## Sec 5.3.1: Evaluation

In [ ]:
names = ["GLM", "CANN", "MDN", "DDR", "DRN"]
models = [glm, cann, mdn, ddr, drn]

print("Generating distributional forecasts")
dists_train = {}
dists_val = {}
dists_test = {}

for name, model in zip(names, models):
    print(f"- {name}")
    dists_train[name] = model.distributions(X_train)
    dists_val[name] = model.distributions(X_val)
    dists_test[name] = model.distributions(X_test)

In [ ]:
print("Calculating CDF over a grid")
GRID_SIZE = 3000  # Increase this to get more accurate CRPS estimates
grid = torch.linspace(0, np.max(y_train) * 1.1, GRID_SIZE).unsqueeze(-1)

cdfs_train = {}
cdfs_val = {}
cdfs_test = {}

for name, model in zip(names, models):
    print(f"- {name}")
    cdfs_train[name] = dists_train[name].cdf(grid)
    cdfs_val[name] = dists_val[name].cdf(grid)
    cdfs_test[name] = dists_test[name].cdf(grid)

### NLL

In [ ]:
print("Calculating negative loglikelihoods")
nlls_train = {}
nlls_val = {}
nlls_test = {}

for name, model in zip(names, models):
    nlls_train[name] = -dists_train[name].log_prob(Y_train).mean()
    nlls_val[name] =-dists_val[name].log_prob(Y_val).mean()
    nlls_test[name] = -dists_test[name].log_prob(Y_test).mean()

for nll_dict, df_name in zip([nlls_train, nlls_val, nlls_test],['training', 'val', 'test']):
    print(f'NLL on {df_name} set')
    for name, model in zip(names, models):
        print(f"{name}: {nll_dict[name]:.4f}")
    print(f'-------------------------------')

In [ ]:
nll_wilcoxon_test(dists_val, Y_val, 'Validation')
nll_wilcoxon_test(dists_test, Y_test, 'Test')

### CRPS

In [ ]:
print("Calculating CRPS")
grid = grid.squeeze()
crps_train ={}
crps_val = {}
crps_test = {}


for name, model in zip(names, models):
    crps_train[name] = crps(Y_train, grid, cdfs_train[name])
    crps_val[name] = crps(Y_val, grid, cdfs_val[name])
    crps_test[name] = crps(Y_test, grid, cdfs_test[name])

for crps_dict, df_name in zip([crps_train, crps_val, crps_test],['training', 'val', 'test']):
    print(f'CRPS on {df_name} set')
    for name, model in zip(names, models):
        print(f"{name}: {crps_dict[name].mean():.4f}")
    print(f'------------------------------')

In [ ]:
crps_wilcoxon_test(cdfs_val, Y_val, grid, 'Validation')
crps_wilcoxon_test(cdfs_test, Y_test, grid, 'Validation')

### RMSE

In [ ]:
rmse_train = {}
rmse_val = {}
rmse_test = {}

for name, model in zip(names, models):
    means_train = dists_train[name].mean
    means_val = dists_val[name].mean 
    means_test = dists_test[name].mean  
    rmse_train[name] = rmse(y_train, means_train)
    rmse_val[name] = rmse(y_val, means_val)
    rmse_test[name] = rmse(y_test, means_test)

for rmse_dict, df_name in zip([rmse_train, rmse_val, rmse_test], ['training', 'validation', 'test']):
    print(f'RMSE on {df_name} set')
    for name, model in zip(names, models):
        print(f"{name}: {rmse_dict[name].mean():.4f}")
    print(f'-------------------------------')

In [ ]:
rmse_wilcoxon_test(dists_val, Y_val, 'Validation')
rmse_wilcoxon_test(dists_test, Y_test, 'Test')

### 90 Quantile Loss

In [ ]:
ql_90_train = {}
ql_90_val = {}
ql_90_test = {}

for features, response, dataset_name, ql_dict in zip(
                            [X_train, X_val, X_test],
                            [y_train, y_val, y_test],
                            ['Training', 'Validation', 'Test'],
                            [ql_90_train, ql_90_val, ql_90_test]):
    print(f'{dataset_name} Dataset Quantile Loss(es)')
    for model, model_name in zip(models, names):
        ql_dict[model_name] = quantile_losses(
                                            0.9,
                                            model,
                                            model_name,
                                            features,
                                            response,
                                            max_iter = 1000,
                                            tolerance = 1e-4,\
                                            l = torch.Tensor([0]),\
                                            u = torch.Tensor([np.max(y_train)+3*(np.max(y_train)-np.min(y_train))])
                                            )
    print(f"----------------------")

In [ ]:
ql90_wilcoxon_test(X_val, Y_val, 'Validation')
ql90_wilcoxon_test(X_test, Y_test, 'Test')

### Table

In [ ]:
def generate_latex_table(nlls_val, crps_val, rmse_val, ql_90_val,\
                         nlls_test, crps_test, rmse_test, ql_90_test,\
                        model_names, label_txt='Evaluation Metrics Table',\
                         caption_txt = 'Evaluation Metrics Table.',\
                           scaling_factor = 1.0):
    header_row = ("\\begin{center}\n"
                +  "\captionof{table}{" + f'{caption_txt}' + '}\n'
                + "\label{" + f'{label_txt}' + '}\n'
                + "\scalebox{" + f'{scaling_factor}' + '}{\n'
                + "\\begin{tabular}{l|cccc|cccc}\n\\toprule\n\\toprule\n"
                + "&  \multicolumn{4}{c}{$\mathcal{D}_{\\text{Validation}}$}"
                + "& \multicolumn{4}{c}{ $\mathcal{D}_{\\text{Test}}$}\\\\ \n"
                + " \cmidrule{2-5}  \cmidrule{6-9} $\\text{Model}$ $\\backslash$ $\\text{Metrics}$"
                + " & NLL & CRPS & RMSE & 90\% QL & NLL & CRPS & RMSE & 90\% QL \\\\ \\midrule")
    rows = [header_row]

    for name in model_names:
        row = ( 
              f"{name} &  {(nlls_val[name].mean()):.4f}" 
              f" &  {(crps_val[name].mean()):.4f} "  
              f" & {(rmse_val[name].mean()):.4f} "  
              f" & {(ql_90_val[name].mean()):.4f} " 
              f" & {(nlls_test[name].mean()):.4f} "
              f" & {(crps_test[name].mean()):.4f} " 
              f" & {(rmse_test[name].mean()):.4f} "  
              f" & {(ql_90_test[name].mean()):.4f} \\\\ "  
              )
        rows.append(row)
    
    table = ("\n".join(rows)  
                  + "\n\\bottomrule\n\\bottomrule" 
                  + "\n\\end{tabular}"
                  +  "\n}"
                  + "\n\end{center}")
    return table


latex_table = generate_latex_table(nlls_val, crps_val, rmse_val, ql_90_val,
                                   nlls_test, crps_test, rmse_test, ql_90_test, names,
                                    label_txt='Evaluation Metrics',
                                    caption_txt = 'Model comparisons based on various evaluation metrics.',
                                    scaling_factor = 0.95 
                                    )
print(latex_table)

### Quantile Residuals

In [ ]:
quantile_residuals_plots(quantile_points(cdfs_test, y_test, grid))
plt.savefig("plots/synth/Quantile Residuals Plot Synthetic.png");
plt.show()

### Calibration

In [ ]:
calibration_plot(cdfs_test, y_test, grid)
plt.savefig("plots/synth/Calibration Plot Synthetic.png");

## Sec 5.3.2: Interpretability

In [ ]:
# Investigated Instance
x_1 = 0.1
x_2 = 0.1

### Local: Density Plot and Kernel SHAP

In [ ]:
# Initialise Explainer
drn_explainer = DRNExplainer(drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, ct)

# Plot adjustment factors
drn_explainer.plot_adjustment_factors(
    instance=pd.DataFrame(np.array([x_1, x_2]).reshape(1, 2), columns=['X_1', 'X_2']),
    num_interpolations=1000,
    plot_adjustments_labels=False,
    x_range=(0, 6),
    synthetic_data=generate_synthetic_gamma_lognormal,
)

plt.savefig("plots/(0.1, 0.1) Density Plot.png")

In [ ]:
drn_explainer = DRNExplainer(drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, column_transformer=ct)

# Plot DP adjustment SHAP for Mean Adjustment Explanation
drn_explainer.plot_dp_adjustment_shap(
    instance_raw=pd.DataFrame(np.array([x_1, x_2]).reshape(1, 2), columns=["X_1", "X_2"]),
    method='Kernel',
    nsamples_background_fraction=0.5,
    top_K_features=3,
    labelling_gap=0.12,
    dist_property='Mean',
    x_range=(2.14, 2.23),
    y_range=(0.0, 0.75),
    observation=True,
    density_transparency=0.9,
    adjustment=True,
    shap_fontsize=15,
    figsize=(7, 7),
    plot_title='Mean Adjustment Explanation',
    synthetic_data=generate_synthetic_gamma_lognormal,
    legend_loc='upper left',
)

plt.savefig("plots/(0.1, 0.1) Mean Adjustment Plot.png")

In [ ]:
drn_explainer = DRNExplainer(
    drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, column_transformer=ct
)

drn_explainer.cdf_plot(
    instance=pd.DataFrame(
        np.array([x_1, x_2]).reshape(1, 2), columns=["X_1", "X_2"]
    ),
    method='Kernel',
    nsamples_background_fraction=0.005,
    top_K_features=3,
    labelling_gap=0.15,
    dist_property='90% Quantile',
    quantile_bounds = (torch.Tensor([cutpoints_DRN[0]]), torch.Tensor([cutpoints_DRN[-1]*2])),
    x_range=(3.4, 3.58),
    y_range=(0.87, 0.93),
    density_transparency=0.9,
    adjustment=True,
    shap_fontsize=15,
    figsize=(7, 7),
    plot_title='90% Quantile Adjustment Explanation',
    synthetic_data=generate_synthetic_gamma_lognormal,
)

plt.savefig("plots/(0.1, 0.1) Quantile Adjustment Plot.png")

In [ ]:
drn_explainer = DRNExplainer(drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, column_transformer=ct)

# Plot CDF for 90% Quantile Explanation
drn_explainer.cdf_plot(
    instance=pd.DataFrame(np.array([x_1, x_2]).reshape(1, 2), columns=["X_1", "X_2"]),
    method='Kernel',
    nsamples_background_fraction=0.05,
    top_K_features=3,
    labelling_gap=0.16,
    dist_property='90% Quantile',
    quantile_bounds = (torch.Tensor([cutpoints_DRN[0]]), torch.Tensor([cutpoints_DRN[-1]*2])),
    x_range=(0, 8),
    y_range=(0., 1.0),
    density_transparency=0.9,
    adjustment=False,
    plot_baseline=False,
    synthetic_data=generate_synthetic_gamma_lognormal,
    shap_fontsize=15,
    figsize=(7, 7),
    plot_title='90% Quantile Explanation',
)

plt.savefig("plots/(0.1, 0.1) Quantile Explanation Plot.png")

### Global: SHAP Dependence

In [ ]:
# Initialise DRNExplainer
if shap_run:
    drn_explainer = DRNExplainer(
        drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, ct
    )

    # Calculate Kernel SHAP values for the DRN model
    kernel_shap_drn = drn_explainer.kernel_shap(
        explaining_data=x_test_raw,
        distributional_property='Mean', #can change to 'XX% Quantile',
        nsamples_background_fraction=0.5,
        adjustment=True,
        glm_output=True
    )

In [ ]:
if shap_run:
    kernel_shap_drn.global_importance_plot(num_features+cat_features, output = 'drn')
    plt.savefig("plots/(Synthetic) SHAP Importance Mean.png");
    plt.show()
    kernel_shap_drn.beeswarm_plot(num_features+cat_features, output = 'drn')
    plt.savefig("plots/(Synthetic) SHAP Beeswarm Mean.png");
    plt.show()
    kernel_shap_drn.shap_dependence_plot(('X_1', 'X_2'), output = 'drn')
    plt.savefig("plots/(Synthetic) SHAP Dependence Mean.png");
    plt.show()

# Section 6: Real Data

## Sec 6.1: Data Preprocessing

In [ ]:
# Read the CSV file into a DataFrame
csv_file_path = 'freMPL1.csv'
df = pd.read_csv(csv_file_path, on_bad_lines='skip')
claims = df.loc[df["ClaimAmount"] > 0, :]

claims["ClaimAmount"].plot.density(color="green", xlim=(0, 30000))
# Setting the title with a larger font size
plt.title('Empirical Density of Truncated Claims', fontsize=20)

# Setting the labels for x and y axes with larger font sizes 
plt.xlabel('Claim Amount ($)', fontsize=15)
plt.ylabel('', fontsize=15)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.savefig("plots/Empirical Density (Real).png")

In [ ]:
# Scaling
target = claims["ClaimAmount"] / 1000
features = claims.drop("ClaimAmount", axis=1)
features = features.drop(
            ["RecordBeg", "RecordEnd", "ClaimInd", "Garage"], axis=1
        )  # Drop garage due to missing values

# Convert "VehAge" categories to numeric
features["VehAge"] = features["VehAge"].map(
            {
                "0": 0,
                "1": 1,
                "2": 2,
                "3": 3,
                "4": 4,
                "5": 5,
                "6-7": 6,
                "8-9": 8,
                "10+": 11,
            }
        )
feature_names = features.columns

speed_ranges = [speed for speed in np.unique(features["VehMaxSpeed"])]
speed_series = pd.Series(speed_ranges)
mapping = {speed_range: i + 1 for i, speed_range in enumerate(speed_ranges)}
features["VehMaxSpeed"] = features["VehMaxSpeed"].map(mapping)
features["SocioCateg"] = features["SocioCateg"].str.extract("(\d+)").astype(int)

cat_features = [
            "HasKmLimit",
            "Gender",
            "MariStat",
            "VehUsage",
            "VehBody",
            "VehPrice",
            "VehEngine",
            "VehEnergy",
            "VehClass",
            "SocioCateg"
        ]

num_features = [
            feature for feature in features.columns if feature not in cat_features
        ]

In [ ]:
# Split and preprocess the data
x_train, x_val, x_test, y_train, y_val, y_test, \
x_train_raw, x_val_raw, x_test_raw, \
num_features, cat_features, \
all_categories, ct = split_and_preprocess(
    features, target, num_features, cat_features, seed=0
)

# Calculate and print statistics for y_train, y_val, y_test
np.max(y_train), np.median(y_train), np.max(y_val), np.median(y_val), np.max(y_test), np.median(y_test)

In [ ]:
# We'll use lowercase "x_train" "y_train" for pandas dataframes
# and uppercase "X_train" "Y_train" for tensors.
X_train = torch.Tensor(x_train.values)
Y_train = torch.Tensor(y_train.values)
X_val = torch.Tensor(x_val.values)
Y_val = torch.Tensor(y_val.values)
X_test = torch.Tensor(x_test.values)
Y_test = torch.Tensor(y_test.values)

train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, Y_val)

## Sec 6.2: Training

In [ ]:
training = False
tuning = False
shap_run = False
distribution = "gamma"

### GLM

In [ ]:
%%time
print("\n\nTraining GLM\n")

torch.manual_seed(23)
glm = GLM(X_train.shape[1], distribution = distribution)
if training:
    train(
        glm,
        gaussian_deviance_loss if distribution == "gaussian" else gamma_deviance_loss,
        train_dataset,
        val_dataset,
        log_interval = 10,
        epochs=5000,
        lr=0.001,
        patience=100,
        batch_size=100,
    )
    glm.update_dispersion(X_train, Y_train)
    glm.eval()
    torch.save(glm.state_dict(), "models/real/glm.pt")
else:
    glm.load_state_dict(torch.load("models/real/glm.pt"))
    glm.eval()

### CANN

In [ ]:
if tuning:
    space_cann_real = [
        Integer(1, 6, name='num_hidden_layers'),
        Categorical([32, 64, 128, 256, 512], name='hidden_size'),
        Real(0.0, 0.5, name='dropout_rate'),
        Real(0.0001, 0.01, name='lr'),
        Categorical([64, 128, 256, 512], name='batch_size'),
    ]

    # Run Bayesian optimization
    res_cann_real = skopt.gp_minimize(
        lambda params: objective_cann(params,
                                    X_train, Y_train, X_val, Y_val, train_dataset, val_dataset, 'gamma'),
        space_cann_real,
        n_calls=200,
        n_random_starts=25,
        random_state=0,
        verbose=True
    )
    best_cann_paras = res_cann_real.x
    print(best_cann_paras)

In [ ]:
%%time
print("\n\nTraining CANN\n")

# Identical hyperparamters from the paper
if not tuning:
    # Format: [hidden layers, neurons/layer,
    #          dropout rate, learning rate, batch size]
    best_cann_paras = [4, 512, 0.43674204603377464, 0.008896658675599278, 256]

torch.manual_seed(23)
cann = CANN(glm, num_hidden_layers=int(best_cann_paras[0]), hidden_size=int(best_cann_paras[1]),
         dropout_rate = best_cann_paras[2])

if training:
    torch.manual_seed(23)
    train(
        cann,
        gaussian_deviance_loss if distribution == "gaussian" else gamma_deviance_loss,
        train_dataset,
        val_dataset,
        epochs=2000,
        lr=best_cann_paras[-2],
        patience=50,
        batch_size=int(best_cann_paras[-1]),
    )
    cann.update_dispersion(X_train, Y_train)
    cann.eval()
    torch.save(cann.state_dict(), "models/real/cann.pt")
else:
    cann.load_state_dict(torch.load("models/real/cann.pt"))
    cann.eval()

### MDN

In [ ]:
if tuning:
    space_mdn_real = [
        Integer(1, 6, name='num_hidden_layers'),
        Categorical([32, 64, 128, 256, 512], name='hidden_size'),
        Real(0.0, 0.5, name='dropout_rate'),
        Real(0.0001, 0.01, name='lr'),
        Integer(2, 10, name='num_components'),
        Categorical([64, 128, 256, 512], name='batch_size'),
    ]

    # Run Bayesian optimization
    res_mdn_real = skopt.gp_minimize(
        lambda params: objective_mdn(params,
                                    X_train, Y_train, X_val, Y_val, train_dataset, val_dataset, 'gamma'),
        space_mdn_real,
        n_calls=200,
        n_random_starts=25,
        random_state=0,
        verbose=True
    )
    best_mdn_paras = res_mdn_real.x
    print(best_mdn_paras)

In [ ]:
%%time
print("\n\nTraining MDN\n")

# Identical hyperparameters from the paper
if not tuning:
    # Format: [hidden layers, neurons/layer,
    #          dropout rate, learning rate,
    #          components, batch size]
    best_mdn_paras = [3, 256, 0.43747364699537183, 0.008452597861700977, 4, 256]
    
torch.manual_seed(23)
mdn = MDN(X_train.shape[1], num_components=int(best_mdn_paras[-2]), hidden_size=int(best_mdn_paras[1]),
            num_hidden_layers=int(best_mdn_paras[0]), dropout_rate = best_mdn_paras[2],\
            distribution= distribution)

if training:  
    torch.manual_seed(23)
    train(
        mdn,
        gaussian_mdn_loss if distribution == "gaussian" else gamma_mdn_loss,
        train_dataset,
        val_dataset,
        lr=best_mdn_paras[3],
        batch_size=int(best_mdn_paras[-1]),
        epochs=2000,
        patience=50,
    )
    mdn.eval()
    torch.save(mdn.state_dict(), "models/real/mdn.pt")
else:
    mdn.load_state_dict(torch.load("models/real/mdn.pt"))
    mdn.eval()

### DDR

In [ ]:
if tuning:
    space_ddr_real = [
        Integer(1, 6, name='num_hidden_layers'),
        Categorical([32, 64, 128, 256, 512], name='hidden_size'),
        Real(0.0, 0.5, name='dropout_rate'),
        Real(0.0002, 0.01, name='lr'),
        Categorical([0.05, 0.075, 0.1, 0.125, 0.15], name='proportion'),
        Categorical([64, 128, 256, 512], name='batch_size'),
    ]

    # Run Bayesian optimization
    res_ddr_real = skopt.gp_minimize(
        lambda params: objective_ddr(params, X_train, Y_train, X_val, Y_val, train_dataset, val_dataset),
        space_ddr_real,
        n_calls=200,
        n_random_starts=25,
        random_state=0,
        verbose=True
    )
    best_ddr_paras = res_ddr_real.x
    print(best_ddr_paras)

In [ ]:
%%time
print("\n\nTraining DDR\n")

# Identical hyperparamters from the paper
if not tuning:
    # Format: [hidden layers, neurons/layer,
    #          dropout rate, learning rate,
    #          cutpoints proportion, batch size]
    best_ddr_paras = [1, 512, 0.5, 0.005757483612427741, 0.15, 256]
    
cutpoints_DDR = ddr_cutpoints(c_0 = np.min(y_train) * 1.05 if np.min(y_train) < 0 else 0.0,
                              c_K = np.max(y_train) * 1.05, 
                              y = y_train,
                              p = best_ddr_paras[-2])

torch.manual_seed(23)
ddr = DDR(X_train.shape[1], cutpoints = cutpoints_DDR, hidden_size=int(best_ddr_paras[1]),
            num_hidden_layers=int(best_ddr_paras[0]), dropout_rate = best_ddr_paras[2])

if training:
    torch.manual_seed(23)
    train(
        ddr,
        ddr_loss,
        train_dataset,
        val_dataset, 
        lr=0.0005,
        batch_size=100,
        log_interval=1,
        patience=30,
        epochs=1000,
    )
    ddr.eval()
    torch.save(ddr.state_dict(), "models/real/ddr.pt")
else:
    ddr.load_state_dict(torch.load("models/real/ddr.pt"))
    ddr.eval()

### DRN

In [ ]:
if tuning:
    space_drn_real = [
        Integer(1, 6, name='num_hidden_layers'),
        Categorical([32, 64, 128, 256, 512], name='hidden_size'),
        Real(0.0, 0.5, name='dropout_rate'),
        Real(0.0002, 0.01, name='lr', prior = 'log-uniform'),
        Real(1e-6, 1e-1, name='kl_alpha', prior = 'log-uniform'),
        Categorical([0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], name='mean_alpha'),
        Categorical([0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], name='dv_alpha'),
        Categorical([64, 128, 256, 512], name='batch_size'),
        Categorical([0.1, 0.125, 0.15], name='proportion'),
        Categorical([1, 3, 5], name='min_observation'),
    ]

    # Run Bayesian optimization
    res_drn_real = skopt.gp_minimize(
        lambda params: objective_drn(params,
                                    X_train=X_train,
                                    Y_train=Y_train,
                                    X_val=X_val,
                                    Y_val=Y_val,
                                    train_dataset=train_dataset,
                                    val_dataset=val_dataset,
                                    patience = 50,
                                    kl_direction= 'forwards'),
        space_drn_real,
        n_calls=200,
        n_random_starts=30,
        random_state=0,
        verbose=True
    )
    best_drn_paras = res_drn_real.x
    print(best_drn_paras)

In [ ]:
%%time
print("\n\nTraining DRN\n")

# Identical hyperparamters from the paper
if not tuning:
  # Format: [hidden layers, neurons/layer,
  #          dropout rate, learning rate,
  #          kl penalty, mean penalty, roughnesss penalty,
  #           batch size, proportion, minimum number of observations]
    best_drn_paras = [2, 512,
                    0.26987049089983844, 0.002907776355380247,
                    0.0016204637505694557, 1e-06, 1e-05,
                    512, 0.125, 3]

cutpoints_DRN = drn_cutpoints(c_0 = np.min(Y_train.detach().numpy()) * 1.05 if np.min(Y_train.detach().numpy()) < 0 else 0.0,
                              c_K = np.max(Y_train.detach().numpy()) * 1.05,
                              p = best_drn_paras[-2],
                              y = Y_train.detach().numpy(),
                              min_obs = int(best_drn_paras[-1]))

torch.manual_seed(23)
drn = DRN(num_features = X_train.shape[1], cutpoints = cutpoints_DRN, glm = glm,\
                    hidden_size=int(best_drn_paras[1]), num_hidden_layers=int(best_drn_paras[0]),
                      baseline_start = False,  dropout_rate = best_drn_paras[2])
if training:
    torch.manual_seed(23)
    train(
            model=drn,
            criterion=lambda pred, y: drn_loss(pred, y, kl_alpha = best_drn_paras[4], #2e-4
                                                mean_alpha = best_drn_paras[5],  #
                                                tv_alpha = 0,
                                                 dv_alpha = best_drn_paras[6]),   
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            batch_size=int(best_drn_paras[7]),
            epochs=2000,
            patience=50,
            lr=best_drn_paras[3],
            print_details=True,
            log_interval=1,
    )
    drn.eval()
    torch.save(drn.state_dict(), "models/real/drn.pt")
else:
    drn.load_state_dict(torch.load("models/real/drn.pt"))
    drn.eval()

## Sec 6.3: Evaluation

In [ ]:
names = ["GLM", "CANN", "MDN", "DDR", "DRN"]
models = [glm, cann, mdn, ddr, drn]

print("Generating distributional forecasts")
dists_train = {}
dists_val = {}
dists_test = {}

for name, model in zip(names, models):
    print(f"- {name}")
    dists_train[name] = model.distributions(X_train)
    dists_val[name] = model.distributions(X_val)
    dists_test[name] = model.distributions(X_test)

print("Calculating CDF over a grid")
GRID_SIZE = 3000  # Increase this to get more accurate CRPS estimates
grid = torch.linspace(0, np.max(y_train) * 1.1, GRID_SIZE).unsqueeze(-1)

cdfs_train = {}
cdfs_val = {}
cdfs_test = {}

for name, model in zip(names, models):
    print(f"- {name}")
    cdfs_train[name] = dists_train[name].cdf(grid)
    cdfs_val[name] = dists_val[name].cdf(grid)
    cdfs_test[name] = dists_test[name].cdf(grid)

### NLL

In [ ]:
print("Calculating negative loglikelihoods")
nlls_train = {}
nlls_val = {}
nlls_test = {}

for name, model in zip(names, models):
    nlls_train[name] = -dists_train[name].log_prob(Y_train).mean()
    nlls_val[name] =-dists_val[name].log_prob(Y_val).mean()
    nlls_test[name] = -dists_test[name].log_prob(Y_test).mean()
    


for nll_dict, df_name in zip([nlls_train, nlls_val, nlls_test],['training', 'val', 'test']):
    print(f'NLL on {df_name} set')
    for name, model in zip(names, models):
        print(f"{name}: {nll_dict[name].mean():.4f}")
    print(f'-------------------------------')

In [ ]:
nll_wilcoxon_test(dists_val, Y_val, 'Validation')
nll_wilcoxon_test(dists_test, Y_test, 'Test')

### CRPS

In [ ]:
print("Calculating CRPS")
grid =grid.squeeze()
crps_train ={}
crps_val = {}
crps_test = {}

for name, model in zip(names, models):
    crps_train[name] = crps(Y_train, grid, cdfs_train[name])
    crps_val[name] = crps(Y_val, grid, cdfs_val[name])
    crps_test[name] = crps(Y_test, grid, cdfs_test[name])

for crps_dict, df_name in zip([crps_train, crps_val, crps_test],['training', 'val', 'test']):
    print(f'CRPS on {df_name} set')
    for name, model in zip(names, models):
        print(f"{name}: {crps_dict[name].mean():.4f}")
    print(f'------------------------------')

In [ ]:
crps_wilcoxon_test(cdfs_val, Y_val, grid, 'Validation')
crps_wilcoxon_test(cdfs_test, Y_test, grid, 'Validation')

### RMSE

In [ ]:
rmse_train = {}
rmse_val = {}
rmse_test = {}

for name, model in zip(names, models):
    means_train = dists_train[name].mean
    means_val = dists_val[name].mean
    means_test = dists_test[name].mean
    rmse_train[name] = rmse(y_train, means_train)
    rmse_val[name] = rmse(y_val, means_val)
    rmse_test[name] = rmse(y_test, means_test)

for rmse_dict, df_name in zip([rmse_train, rmse_val, rmse_test], ['training', 'validation', 'test']):
    print(f'RMSE on {df_name} set')
    for name, model in zip(names, models):
        print(f"{name}: {rmse_dict[name].mean():.4f}")
    print(f'-------------------------------')

In [ ]:
rmse_wilcoxon_test(dists_val, Y_val, 'Validation')
rmse_wilcoxon_test(dists_test, Y_test, 'Test')

### 90 Quantile Loss

In [ ]:
ql_90_train = {}
ql_90_val = {}
ql_90_test = {}

for features, response, dataset_name, ql_dict in zip(
                            [X_train, X_val, X_test],
                            [y_train, y_val, y_test],
                            ['Training', 'Validation', 'Test'],
                            [ql_90_train, ql_90_val, ql_90_test]):
    print(f'{dataset_name} Dataset Quantile Loss(es)')
    for model, model_name in zip(models, names):
        ql_dict[model_name] = quantile_losses(
                                            0.9,
                                            model,
                                            model_name,
                                            features,
                                            response,
                                            max_iter = 1000,
                                            tolerance = 1e-4,\
                                            l = torch.Tensor([0]),\
                                            u = torch.Tensor([np.max(y_train)+3*(np.max(y_train)-np.min(y_train))])
                                            )
    print(f"----------------------")

In [ ]:
ql90_wilcoxon_test(X_val, Y_val, 'Validation')
ql90_wilcoxon_test(X_test, Y_test, 'Test')

### Table

In [ ]:
def generate_latex_table(nlls_val, crps_val, rmse_val, ql_90_val,\
                         nlls_test, crps_test, rmse_test, ql_90_test,\
                        model_names, label_txt='Evaluation Metrics Table',\
                         caption_txt = 'Evaluation Metrics Table.',\
                           scaling_factor = 1.0):
    header_row = ("\\begin{center}\n"
                +  "\captionof{table}{" + f'{caption_txt}' + '}\n'
                + "\label{" + f'{label_txt}' + '}\n'
                + "\scalebox{" + f'{scaling_factor}' + '}{\n'
                + "\\begin{tabular}{l|cccc|cccc}\n\\toprule\n\\toprule\n"
                + "&  \multicolumn{4}{c}{$\mathcal{D}_{\\text{Validation}}$}"
                + "& \multicolumn{4}{c}{ $\mathcal{D}_{\\text{Test}}$}\\\\ \n"
                + " \cmidrule{2-5}  \cmidrule{6-9} $\\text{Model}$ $\\backslash$ $\\text{Metrics}$"
                + " & NLL & CRPS & RMSE & 90\% QL & NLL & CRPS & RMSE & 90\% QL \\\\ \\midrule")
    rows = [header_row]

    for name in model_names:
        row = ( 
              f"{name} &  {(nlls_val[name].mean()):.4f}" 
              f" &  {(crps_val[name].mean()):.4f} "  
              f" & {(rmse_val[name].mean()):.4f} "  
              f" & {(ql_90_val[name].mean()):.4f} " 
              f" & {(nlls_test[name].mean()):.4f} "
              f" & {(crps_test[name].mean()):.4f} " 
              f" & {(rmse_test[name].mean()):.4f} "  
              f" & {(ql_90_test[name].mean()):.4f} \\\\ "  
              )
        rows.append(row)
    
    table = ("\n".join(rows)  
                  + "\n\\bottomrule\n\\bottomrule" 
                  + "\n\\end{tabular}"
                  +  "\n}"
                  + "\n\end{center}")
    return table


latex_table = generate_latex_table(nlls_val, crps_val, rmse_val, ql_90_val,
                                   nlls_test, crps_test, rmse_test, ql_90_test, names,
                                    label_txt='Evaluation Metrics',
                                    caption_txt = 'Model comparisons based on various evaluation metrics.',
                                    scaling_factor = 0.95 
                                    )
print(latex_table)

### Quantile Residuals

In [ ]:
quantile_residuals_plots(quantile_points(cdfs_test, y_test, grid))
plt.savefig("plots/real/Quantile Residuals Plot Real.png");

### Calibration

In [ ]:
calibration_plot(cdfs_test, y_test, grid)
plt.savefig("plots/real/Calibration Plot Real.png");

## Sec 6.4: Interpretability

### Sec 6.4.1 Local Interpretability

#### (a) Extreme Case

In [ ]:
means_diff = drn.distributions(X_test).mean - glm.distributions(X_test).mean
# Find the top 5 values and their indices
values, indices = torch.topk(means_diff.view(-1), 3, sorted=True)
multi_indices = np.unravel_index(indices.numpy(), means_diff.shape)

print(multi_indices, means_diff[multi_indices])
idx_first = multi_indices[0][0]
idx_second = multi_indices[0][1]
idx_third = multi_indices[0][2]
y_test.values[multi_indices], drn.distributions(X_test).mean[multi_indices], glm.distributions(X_test).mean[multi_indices],

In [ ]:
drn_explainer = DRNExplainer(drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, column_transformer=ct)  
idx = idx_second
drn_explainer.plot_dp_adjustment_shap(instance_raw = x_test_raw.iloc[idx:(idx+1)],\
                                       method = 'Kernel',\
                                       nsamples_background_fraction= 0.5, \
                                       top_K_features= 5,\
                                      labelling_gap = 0.1,\
                                        dist_property= 'Mean',
                                        x_range = (0.0, 50.0),
                                        y_range = (0.0, 0.75),
                                        observation = Y_test[idx:(idx+1)],
                                        density_transparency = 0.5,
                                        adjustment= True,
                                          shap_fontsize=15,
                                          figsize = (7, 7),
                                        plot_title = 'Explaining a Large Mean Adjustment',
                                         legend_loc = 'upper left'
                                          )

plt.savefig("plots/(Real) Mean Adjustment SHAP.png");

In [ ]:
drn_explainer = DRNExplainer(drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, column_transformer=ct)  
idx = idx_second
drn_explainer.plot_dp_adjustment_shap(instance_raw = x_test_raw.iloc[idx:(idx+1)],\
                                       method = 'Kernel',\
                                       nsamples_background_fraction= 0.5, \
                                       top_K_features= 5,\
                                      labelling_gap=0.1,\
                                        dist_property= 'Mean',
                                        #other_df_models = [mdn, ddr], model_names = ["MDN", "DDR"],\
                                        x_range = (0.0, 50.0),
                                        y_range = (0.0, 0.75),
                                        observation = Y_test[idx:(idx+1)],
                                        density_transparency = 0.5,
                                        adjustment= False,
                                          shap_fontsize=15,
                                         figsize = (7, 7),
                                        plot_title = 'Explaining a Large Mean Prediction',
                                         legend_loc = 'upper left'
                                          )

plt.savefig("plots/(Real) Mean Explanation SHAP.png");

#### (b) Average Case

In [ ]:
# Calculate the mean differences between DRN and GLM predictions
means_diff = drn.distributions(X_test).mean - glm.distributions(X_test).mean

# Find indices where the percentage change between means is betweenn than 30% and 50%
valid_indices =  (0.4 < (torch.abs(means_diff) / glm.distributions(X_test).mean)) &  \
                 (0.8 > (torch.abs(means_diff) / glm.distributions(X_test).mean)) 

# Filter X_test, x_test_raw, and y_test based on these valid indices
X_test_new = X_test[valid_indices]
x_test_raw_new = x_test_raw.iloc[valid_indices.numpy()]
y_test_new = Y_test[valid_indices]

# Recalculate mean differences with the filtered dataset
means_diff_new = drn.distributions(X_test_new).mean - glm.distributions(X_test_new).mean
glm_means_new = glm.distributions(X_test_new).mean

# Find instances where the GLM predictions are close to the actual y values
y_diff = torch.abs(y_test_new - glm_means_new)
close_y_indices = y_diff < 0.3 * torch.abs(y_test_new)  # Assuming 30% closeness threshold

# Filter further based on the closeness of GLM predictions to y_test
X_test_final = X_test_new[close_y_indices]
means_diff_final = means_diff_new[close_y_indices]
y_test_final = y_test_new[close_y_indices]
glm_means_final = glm_means_new[close_y_indices]
drn_means_final = drn.distributions(X_test_final).mean

# Ensure we have enough data points after filtering
if len(means_diff_final) >= 4:
    # Find the top 4 values and their indices based on the filtered dataset
    values, indices = torch.topk(torch.abs(means_diff_final).view(-1), 4, largest=False)

    # Extract the original indices for the closest Four instances
    original_indices = valid_indices.nonzero(as_tuple=True)[0][close_y_indices][indices]

    idx_first = original_indices[0].item()
    idx_second = original_indices[1].item()
    idx_third = original_indices[2].item()
    idx_forth = original_indices[3].item()

    # Output the results for the selected instances
    print("Original Indices:", idx_first, idx_second)
    print("Actual y values:", y_test.values[idx_first], y_test.values[idx_second])
    print("DRN mean predictions:", drn.distributions(X_test).mean[idx_first].item(), drn.distributions(X_test).mean[idx_second].item())
    print("GLM mean predictions:", glm.distributions(X_test).mean[idx_first].item(), glm.distributions(X_test).mean[idx_second].item())
else:
    print("Not enough data points meet the criteria.")

In [ ]:
drn_explainer = DRNExplainer(drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, column_transformer=ct)  
idx = idx_first
drn_explainer.plot_dp_adjustment_shap(instance_raw = x_test_raw.iloc[idx:(idx+1)],\
                                       method = 'Kernel',\
                                       nsamples_background_fraction= 0.5, \
                                       top_K_features= 5,\
                                      labelling_gap=0.1,\
                                        dist_property= 'Mean',
                                        #other_df_models = [mdn, ddr], model_names = ["MDN", "DDR"],\
                                        x_range = (0.0, 6.5),
                                        y_range = (0.0, 2.5),
                                        observation = Y_test[idx:(idx+1)],
                                        density_transparency = 0.5,
                                        adjustment= True,
                                          shap_fontsize=15,
                                         figsize = (7, 7),
                                        plot_title = 'Explaining an Average Mean Adjustment',
                                         legend_loc = 'upper left'
                                          )

plt.savefig("plots/(Real) Average Mean Adjustment SHAP.png");

#### (c) Mild Case

In [ ]:
means_diff = drn.distributions(X_test).mean - glm.distributions(X_test).mean

valid_indices =  (0.1 < (torch.abs(means_diff) / glm.distributions(X_test).mean)) &  \
                 (0.3 > (torch.abs(means_diff) / glm.distributions(X_test).mean))

X_test_new = X_test[valid_indices]
x_test_raw_new = x_test_raw.iloc[valid_indices.numpy()]
y_test_new = Y_test[valid_indices]

means_diff_new = drn.distributions(X_test_new).mean - glm.distributions(X_test_new).mean
glm_means_new = glm.distributions(X_test_new).mean

y_diff = torch.abs(y_test_new - glm_means_new)
close_y_indices = y_diff < 0.2 * torch.abs(y_test_new)  # Assuming 20% closeness threshold

X_test_final = X_test_new[close_y_indices]
means_diff_final = means_diff_new[close_y_indices]
y_test_final = y_test_new[close_y_indices]
glm_means_final = glm_means_new[close_y_indices]
drn_means_final = drn.distributions(X_test_final).mean

if len(means_diff_final) >= 4:
    values, indices = torch.topk(torch.abs(means_diff_final).view(-1), 4, largest=False)
    original_indices = valid_indices.nonzero(as_tuple=True)[0][close_y_indices][indices]

    idx_first = original_indices[0].item()
    idx_second = original_indices[1].item()
    idx_third = original_indices[2].item()
    idx_forth = original_indices[3].item()

    # Output the results for the selected instances
    print("Original Indices:", idx_first, idx_second)
    print("Actual y values:", y_test.values[idx_first], y_test.values[idx_second])
    print("DRN mean predictions:", drn.distributions(X_test).mean[idx_first].item(), drn.distributions(X_test).mean[idx_second].item())
    print("GLM mean predictions:", glm.distributions(X_test).mean[idx_first].item(), glm.distributions(X_test).mean[idx_second].item())
else:
    print("Not enough data points meet the criteria.")

In [ ]:
drn_explainer = DRNExplainer(drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, column_transformer=ct)  
idx = idx_second
drn_explainer.plot_dp_adjustment_shap(instance_raw = x_test_raw.iloc[idx:(idx+1)],\
                                       method = 'Kernel',\
                                       nsamples_background_fraction= 0.5, \
                                       top_K_features= 5,\
                                      labelling_gap=0.1,\
                                        dist_property= 'Mean',
                                        #other_df_models = [mdn, ddr], model_names = ["MDN", "DDR"],\
                                        x_range = (0.0, 6.0),
                                        y_range = (0.0, 2.0),
                                        observation = Y_test[idx:(idx+1)],
                                        density_transparency = 0.5,
                                        adjustment= True,
                                          shap_fontsize=15,
                                         figsize = (7, 7),
                                        plot_title = 'Explaining a Mild Mean Adjustment',
                                         legend_loc = 'upper left'
                                          )

plt.savefig("plots/(Real) Mild Mean Adjustment SHAP.png");

#### CDF Plot

In [ ]:
drn_explainer = DRNExplainer(
    drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, column_transformer=ct
)

idx = idx_first
drn_explainer.cdf_plot(
    instance = x_test_raw.iloc[idx:(idx+1)],
    method='Kernel',
    nsamples_background_fraction=0.2,
    top_K_features=5,
    labelling_gap=0.1,
    dist_property='90% Quantile',
    quantile_bounds = (torch.Tensor([cutpoints_DRN[0]]), torch.Tensor([cutpoints_DRN[-1]])),
    x_range=(0.0, 8.5),
    y_range=(0., 1.0),
    density_transparency=0.9,
    adjustment=True,
    shap_fontsize=15,
    figsize=(7, 7),
    plot_title='90% Quantile Adjustment Explanation',
)

### Sec 6.4.2: Global Interpretability

In [ ]:
if shap_run:
  drn_explainer = DRNExplainer(drn, glm, cutpoints_DRN, x_train_raw, cat_features, all_categories, ct)  
  kernel_shap_drn = drn_explainer.kernel_shap(
                                              explaining_data = x_test_raw,
                                              distributional_property= 'Mean',
                                              nsamples_background_fraction = 0.2,
                                              adjustment= True,
                                              glm_output= True) 

In [ ]:
if shap_run:
    kernel_shap_drn.global_importance_plot(num_features+cat_features, output = 'drn')
    plt.tight_layout() 
    plt.savefig("plots/(Real) Global Importance.png");
    plt.show()
    plt.close()
    kernel_shap_drn.beeswarm_plot(num_features+cat_features, output = 'drn')
    plt.tight_layout() 
    plt.savefig("plots/(Real) Beeswarm Summary Plot.png");
    plt.show()
    plt.close()
    kernel_shap_drn.shap_dependence_plot(('MariStat', 'Exposure'), output = 'drn')
    plt.tight_layout() 
    plt.savefig("plots/(SHAP Dependence) MariState X Exposure.png");
    plt.show()
    plt.close()
    kernel_shap_drn.shap_dependence_plot(('MariStat', 'LicAge'), output = 'drn')
    plt.tight_layout() 
    plt.savefig("plots/(SHAP Dependence) MariState X LicAge.png");
    plt.show()
    plt.close()

In [ ]:
if shap_run:
    kernel_shap_drn.global_importance_plot(num_features+cat_features, output = 'value')
    plt.savefig("plots/(Real Adjustment) Global Importance.png");
    plt.show()
    plt.close()
    kernel_shap_drn.beeswarm_plot(num_features+cat_features, output = 'value')
    plt.savefig("plots/(Real Adjustment) Beeswarm Summary Plot.png");
    plt.show()
    plt.close()

In [ ]:
if shap_run:
    kernel_shap_drn.shap_dependence_plot(('VehEnergy', 'Exposure'), output = 'value')
    plt.savefig("plots/(SHAP Adjustment) VehEnergy X Exposure.png");
    plt.show()
    plt.close()
    kernel_shap_drn.shap_dependence_plot(('RiskVar', 'Exposure'), output = 'value')
    plt.savefig("plots/(SHAP Adjustment) RiskVar X Exposure.png");
    plt.show()
    plt.close()
    kernel_shap_drn.shap_dependence_plot(('LicAge', 'DrivAge'), output = 'value')
    plt.savefig("plots/(SHAP Adjustment) LicAge X DrivAge.png");
    plt.show()
    plt.close()